### This code gets expectation data from Brazilian Central Bank - FOCUS database

#### References: 
- Filho, L. H. B. (2022, April 11). Coletando dados do Banco Central com Python. Análise Macro. https://analisemacro.com.br/economia/indicadores/coletando-dados-do-banco-central-com-python/.
Contribution: The basic approach to access Brazilian Central Bank API.

#### Libraries:
- Freitas, W. (2023). Python-bcb (0.2.0) [Python]. https://pypi.org/project/python-bcb/ - Freitas, W. (2021). Documentação python-bcb. Documentação python-bcb. https://wilsonfreitas.github.io/python-bcb/
- Package Pandas (2.2). (2024). [Python]. https://pandas.pydata.org/

In [1]:
# Import libraries

import pandas as pd
from bcb import Expectativas

import useful_functions as uf

In [2]:
# Get info from "Expectativas" API
expec = Expectativas()

# get the available endpoints
#expec.describe()

# Check endpoint
expec.describe('ExpectativasMercadoAnuais')



EntitySet (Endpoint): ExpectativasMercadoAnuais
EntityType: br.gov.bcb.olinda.servico.Expectativas.ExpectativaMercadoAnual
Properties: Indicador<str>, IndicadorDetalhe<str>, Data<str>, DataReferencia<str>, Media<float>, Mediana<float>, DesvioPadrao<float>, Minimo<float>, Maximo<float>, numeroRespondentes<int>, baseCalculo<int>


In [3]:
# Define parameters to query BCB server

# Define the start date - yyyy/mm/dd
start_date = '2001-01-01'

# Import the endpoint
ep = expec.get_endpoint('ExpectativasMercadoAnuais')
# Query the endpoint
df_raw = ep.query().filter(ep.Data >= start_date).collect()



In [4]:
# Define the targeted indicators
target_indicators = ['Balança comercial', 
                       'Investimento direto no país', 
                       'Resultado primário', 
                       'IPCA', 
                       'PIB Total',
                       'Selic', 
                       'Câmbio', 
                       'Dívida bruta do governo geral', 
                       'Dívida líquida do setor público'
                       ]

# Filter the targeted indicators
filtered_df = df_raw.loc[df_raw['Indicador'].isin(target_indicators)]

# Filter forecasts made during 30 days before publication
filtered_df = filtered_df.loc[filtered_df['baseCalculo'] == 0] # baseCalculo = 0 only keeps forecasts made during 30 days before publication

# Filter the dataset to not include exports and imports
filtered_df = filtered_df.loc[~filtered_df['IndicadorDetalhe'].isin(['Importações', 'Exportações'])]


In [5]:
# Create a copy
df_BR_indicators = filtered_df.copy()

# In the case of the indicador "Balança comercial", filter the "IndicadorDetalhe" = 'saldo'
df_BR_indicators = df_BR_indicators[~df_BR_indicators['IndicadorDetalhe'].isin(['Exportações', 'Importações'])] # Only keeps the "saldo" of the "Balança comercial" indicator or NAs

# Define the index
df_BR_indicators.set_index('Data', inplace=True)

# Define columns to be dropped
columns_to_drop = ['IndicadorDetalhe',
                   'Media', 
                   'DesvioPadrao', 
                   'Minimo', 
                   'Maximo', 
                   'numeroRespondentes', 
                   'baseCalculo'] # We will use only the median expectations and discard the mean, maximun and minimum.

#Drop the variables
df_BR_indicators.drop(columns_to_drop, axis=1, inplace=True)

# Rename columns
df_BR_indicators.rename(columns={
    'Indicador': 'indicator',
    'DataReferencia': 'reference_year',
    'Mediana': 'median'},inplace=True)

df_BR_indicators.rename_axis('date', inplace=True)

# Convert reference_year to date format
df_BR_indicators['reference_year'] = pd.to_datetime(df_BR_indicators['reference_year'], format='%Y').dt.year

### Apply some filters
df_BR_indicators.index = pd.to_datetime(df_BR_indicators.index)

# Filter reference year with a maximum of two years beyond the collection date
df_BR_indicators = df_BR_indicators[(df_BR_indicators['reference_year'] <= df_BR_indicators.index.year+2)]

### Translate observations:
# Create mapping pattern
indicator_mapping = {
    'Balança comercial': 'trade_balance', #US$ billion
    'Investimento direto no país': 'DIC', #Direct investment in the country - US$ billion
    'Resultado primário': 'primary_result', # % GDP
    'IPCA': 'inflation', # % per year
    'PIB Total': 'GDP (%)', #% of previous year GDP
    'Selic': 'basic_interest_rate', # % per year
    'Câmbio': 'US$_currency', # R$ / US$
    'Dívida bruta do governo geral': 'gross_public_debt', #Gross general government debt  - % GDP
    'Dívida líquida do setor público': 'net_public_debt' #Net public sector debt - % GDP
}

#Aply translation pattern
df_BR_indicators['indicator'] = df_BR_indicators['indicator'].replace(indicator_mapping)
df_BR_indicators.reset_index(inplace=True) # Reset index

# Categorize each reference_year as "y", "y+1", "y+2" instead of the year thus we can compare the forecasts
df_BR_indicators['reference_year'] = df_BR_indicators.apply(lambda row: 'y' 
                            if row['reference_year'] == row['date'].year 
                            else ('y+1' if row['reference_year'] == row['date'].year + 1 
                                  else ('y+2' if row['reference_year'] == row['date'].year + 2 else None)), axis=1)

# Drop years with "None" in the reference_year variable, because they are 'forecasts' for passed years.
df_BR_indicators.dropna(subset=['reference_year'], inplace=True)

# Create a new column with the parameter name to aggregate indicator and reference_year
df_BR_indicators['parameter'] = 'EXPEC_'+ df_BR_indicators['indicator'] + '_' + df_BR_indicators['reference_year'].astype(str)
df_BR_indicators.drop(['indicator', 'reference_year'], axis=1, inplace=True)



In [7]:
# Create a pivot table with the median values of each parameter
df_BR_indicators_wide = df_BR_indicators.pivot(index='date', columns='parameter', values='median')

# Rename the index
df_BR_indicators_wide.rename_axis('Date', inplace=True)

# Convert the dataset to monthly frequency getting the last valid value of each month
df_focus_indicators_monthly = uf.convert_df_to_monthly(df_BR_indicators_wide,'%Y-%m-%d')

# Export the dataset to csv
df_focus_indicators_monthly.to_csv('../data/df_FOCUS.csv', index=True)
